# Utility functions inspired by IPython magics

[IPython magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) is special command available in IPython kernel.
The magic provides helpful functionality such as calling external commands (bash, ruby, etc.), benchmarking, capturing of stdout/stderr, and so on.
The syntax of the magics is different from the Python core language: they starts with `%` or `%%`, and arguments are written like `--param`.

OCaml Jupyter 2.8.0+ supports several utility functions inspired by IPython magics.
Unlike IPython magics, the functions are implemented by the OCaml core language, not syntax extension.
You don't need to study new syntax, and distinguish cell magics and line magics.
The utility functions inherits convenience and flexibility of the OCaml language.
They are included in `jupyter.notebook` package. You can see [API documentation of jupyter.notebook](https://akabe.github.io/ocaml-jupyter/api/jupyter/Jupyter_notebook/) for details.

For example, you can measure execution time of a snippet by the cell magic `%%timeit` in IPython as follows.

```python
%%timeit
2 ** 1000
# > 884 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
```

In OCaml Jupyter, `Jupyter_notebook.Bench.timeit` has the same functionality as `%%timeit`:

```ocaml
open Jupyter_notebook
Bench.timeit (fun () -> 2. ** 1000.)
(* > - : Jupyter_notebook.Bench.stat Jupyter_notebook.Bench.t =
 * >     Jupyter_notebook.Bench.({
 * >      b_rtime = 25.639 ns ± 3.523 ns; b_utime = 25.518 ns ± 3.486 ns;
 * >      b_stime = 0.053 ns ± 0.037 ns; }) *)
```

In [1]:
#require "jupyter.notebook" ;;
open Jupyter_notebook ;;

## 1. Benchmark

Benchmark functions are in `Jupyter_notebook.Bench`.
We provide `time` and `timeit` corresponding to `%%time` and `%%timeit`, respectively.

[Bench.time](https://akabe.github.io/ocaml-jupyter/api/jupyter/Jupyter_notebook/Bench/index.html#val-time) evaluates a function once, and reports time of the execution.
The first component of a return value is `f ()`, and the second is benchmark result:

- `b_rtime` is real time
- `b_utime` is user time
- `b_stime` is sys time

In [2]:
Bench.time (fun () -> 2. ** 1000.)

- : float * float Jupyter_notebook.Bench.t =
(1.07150860718626732e+301, Jupyter_notebook.Bench.({
   b_rtime = 4.053 us; b_utime = 2.000 us; b_stime = 3.000 us; }))


[Bench.timeit](https://akabe.github.io/ocaml-jupyter/api/jupyter/Jupyter_notebook/Bench/index.html#val-timeit) is very similar to `Bench.time`, but the former calls a function repeatedly, and calculates mean of execution time of them.
`timeit` is a little reliable then `time`.

In [3]:
Bench.timeit (fun () -> 2. ** 1000.)

- : Jupyter_notebook.Bench.stat Jupyter_notebook.Bench.t =
Jupyter_notebook.Bench.({
  b_rtime = 26.035 ns ± 3.516 ns; b_utime = 25.773 ns ± 3.352 ns;
  b_stime = 0.132 ns ± 0.081 ns; })


## 2. Subprocess

### 2.1. Command bindings

You can call commands and executable programs from OCaml Jupyter.
[Jupyter_notebook.Process](https://akabe.github.io/ocaml-jupyter/api/jupyter/Jupyter_notebook/Process/index.html) contains easy-to-use bindings of some popular commands.

For example, `ls` command corresponds to `Process.ls`.

In [4]:
Process.ls ["."]

Untitled.ipynb
datasets
install_ocaml_colab.ipynb
introduction.ipynb
utility_like_ipython_magic.ipynb
word_description_from_DuckDuckGoAPI.ipynb


- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0; stdout = None;
 stderr = None}


`sh` is also available.

In [5]:
Process.sh "opam install -y lacaml"

[NOTE] It seems you have not updated your repositories for a while. Consider updating them with:
       opam update



The following actions will be performed:
  ∗ install conf-lapack 1      [required by lacaml]
  ∗ install conf-blas   1      [required by lacaml]
  ∗ install lacaml      11.0.8
===== ∗ 3 =====

<><> Gathering sources ><><><><><><><><><><><><><><><><><><><><><><><><><><>  🐫 
[lacaml.11.0.8] downloaded from cache at https://opam.ocaml.org/cache

<><> Processing actions <><><><><><><><><><><><><><><><><><><><><><><><><><>  🐫 
∗ installed conf-lapack.1
∗ installed conf-blas.1
∗ installed lacaml.11.0.8
Done.


- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0; stdout = None;
 stderr = None}


The long string syntax `{| ... |}` in OCaml is useful for writing a program of several lines.

In [6]:
Process.python3 {|
def greeting(name: str) -> str:
    return 'Hello ' + name

print(greeting('Alice'))|}

Hello Alice


- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0; stdout = None;
 stderr = None}


### 2.2. Capturing stdout/stderr

`capture_stdout` and `capture_stderr` options controls capturing of stdout and stderr in a subprocess.
You can obtain stdout of a command in a return value by passing `~capture_stdout:true`:

In [7]:
Process.sh ~capture_stdout:true {|echo "This text is printed in a shell script."|}

- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0;
 stdout = Some "This text is printed in a shell script.\n"; stderr = None}


``~capture_stderr:`Yes`` or ``~capture_stderr:`Redirect_to_stdout`` also captures stderr of a subprocess.

[Process.capture_in_process](https://akabe.github.io/ocaml-jupyter/api/jupyter/Jupyter_notebook/Process/index.html#val-capture_in_process) captures stdout and stderr of arbitrary user functions.

In [8]:
Process.capture_in_process (fun () -> print_endline "This text is printed in a function.")

- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0;
 stdout = Some "This text is printed in a function.\n"; stderr = None}
